<a href="https://colab.research.google.com/github/GloriaMoraaRiechi/CSE620/blob/main/Untitled8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id, col, when, isnan, count
from pyspark.ml.feature import Imputer, VectorAssembler, StringIndexer, PCA
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


In [2]:
# Initialize Spark session
spark = SparkSession.builder.appName("MulticlassClassification").getOrCreate()

Load data

In [6]:
from google.colab import files

# Upload multiple files
uploaded = files.upload()

# Extract file names
file_names = list(uploaded.keys())

# Ensure correct paths by prepending "/content/"
file_paths = [f"/content/{name}" for name in file_names]

# Read files into separate DataFrames
train_data = spark.read.csv(file_paths[0], header=False, inferSchema=True, sep="\t")
train_label = spark.read.csv(file_paths[1], header=False, inferSchema=True, sep="\t")

# Display the first few rows
train_data.show()
train_label.show()


Saving TrainData1.txt to TrainData1 (2).txt
Saving TrainLabel1.txt to TrainLabel1 (2).txt
+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+--------

In [9]:
train_data.printSchema()
train_label.printSchema()

root
 |-- _c0: double (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: double (nullable = true)
 |-- _c8: double (nullable = true)
 |-- _c9: double (nullable = true)
 |-- _c10: double (nullable = true)
 |-- _c11: double (nullable = true)
 |-- _c12: double (nullable = true)
 |-- _c13: double (nullable = true)
 |-- _c14: double (nullable = true)
 |-- _c15: double (nullable = true)
 |-- _c16: double (nullable = true)
 |-- _c17: double (nullable = true)
 |-- _c18: double (nullable = true)
 |-- _c19: double (nullable = true)
 |-- _c20: double (nullable = true)
 |-- _c21: double (nullable = true)
 |-- _c22: double (nullable = true)
 |-- _c23: double (nullable = true)
 |-- _c24: double (nullable = true)
 |-- _c25: double (nullable = true)
 |-- _c26: double (nullable = true)
 |-- _c27: double (nullable = tru

In [11]:
from pyspark.sql.functions import monotonically_increasing_id, col, when, count
from pyspark.ml.feature import Imputer

# Rename label column before joining
train_label = train_label.withColumnRenamed("_c0", "label")

# Add an ID column to both DataFrames
train_data = train_data.withColumn("id", monotonically_increasing_id())
train_label = train_label.withColumn("id", monotonically_increasing_id())

# Join DataFrames on "id"
train_combined = train_data.join(train_label, on="id", how="inner").drop("id")

# Show the schema to verify correct column names
train_combined.printSchema()


root
 |-- _c0: double (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: double (nullable = true)
 |-- _c8: double (nullable = true)
 |-- _c9: double (nullable = true)
 |-- _c10: double (nullable = true)
 |-- _c11: double (nullable = true)
 |-- _c12: double (nullable = true)
 |-- _c13: double (nullable = true)
 |-- _c14: double (nullable = true)
 |-- _c15: double (nullable = true)
 |-- _c16: double (nullable = true)
 |-- _c17: double (nullable = true)
 |-- _c18: double (nullable = true)
 |-- _c19: double (nullable = true)
 |-- _c20: double (nullable = true)
 |-- _c21: double (nullable = true)
 |-- _c22: double (nullable = true)
 |-- _c23: double (nullable = true)
 |-- _c24: double (nullable = true)
 |-- _c25: double (nullable = true)
 |-- _c26: double (nullable = true)
 |-- _c27: double (nullable = tru

In [12]:
# Get feature columns excluding the label
feature_cols = [c for c in train_combined.columns if c != "label"]

# Count missing values
missing_counts = train_combined.select([
    count(when(col(c).isNull(), c)).alias(c) for c in feature_cols
]).collect()

# If there are missing values, apply Imputer
if any(row[c] > 0 for row in missing_counts for c in row.asDict()):
    imputer = Imputer(inputCols=feature_cols, outputCols=feature_cols)
    train_combined = imputer.fit(train_combined).transform(train_combined)

train_combined.show()


+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+----------------+-------------

In [13]:
categorical_columns = [f.name for f in train_combined.schema.fields if str(f.dataType) == "StringType"]
print("Categorical Columns:", categorical_columns)


Categorical Columns: []


In [15]:
from pyspark.ml.feature import VectorAssembler

# List of feature columns after imputation (usually, these are the same as the previous feature columns)
feature_cols = [c for c in train_combined.columns if c != "label"]

# Assemble features into a single vector column
vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
train_combined = vector_assembler.transform(train_combined).select("features", "label")


In [16]:
# Split the data into training (80%) and test (20%) sets
train_data, test_data = train_combined.randomSplit([0.8, 0.2], seed=42)

# Show the split
train_data.show(5)
test_data.show(5)


+--------------------+-----+
|            features|label|
+--------------------+-----+
|[3.71907096381171...|    1|
|[3.7509077929213,...|    2|
|[3.75863867004645...|    1|
|[3.76405352593726...|    3|
|[3.76459079398616...|    1|
+--------------------+-----+
only showing top 5 rows

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[3.755172559686,1...|    3|
|[3.77180200316728...|    1|
|[3.78468004329535...|    3|
|[3.79997567134642...|    1|
|[3.80982425069004...|    1|
+--------------------+-----+
only showing top 5 rows



In [20]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Define models for multiclass classification
models = {
    "Logistic Regression": LogisticRegression(family="multinomial", featuresCol="features", labelCol="label"),
    "Random Forest": RandomForestClassifier(numTrees=50, featuresCol="features", labelCol="label"),
    "Decision Tree": DecisionTreeClassifier(featuresCol="features", labelCol="label")
}

# Initialize the evaluator for multiclass classification
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

# Train each model and evaluate
for model_name, model in models.items():
    print(f"Training {model_name}...")

    # Train the model
    trained_model = model.fit(train_data)

    # Make predictions on test data
    predictions = trained_model.transform(test_data)

    # Evaluate the model
    accuracy = evaluator.evaluate(predictions)
    print(f"{model_name} Accuracy: {accuracy:.4f}")


Training Logistic Regression...
Logistic Regression Accuracy: 0.4211
Training Random Forest...
Random Forest Accuracy: 0.8421
Training Naive Bayes...
Naive Bayes Accuracy: 0.0000
Training Decision Tree...
Decision Tree Accuracy: 0.7895


In [25]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import numpy as np

# Define models for multiclass classification
models = {
    "Logistic Regression": LogisticRegression(family="multinomial", featuresCol="features", labelCol="label"),
    "Random Forest": RandomForestClassifier(numTrees=50, featuresCol="features", labelCol="label"),
    "Decision Tree": DecisionTreeClassifier(featuresCol="features", labelCol="label")
}

# Initialize the evaluator for PySpark's accuracy metric
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

# Train each model and evaluate
for model_name, model in models.items():
    print(f"Training {model_name}...")

    # Train the model
    trained_model = model.fit(train_data)

    # Make predictions on test data
    predictions = trained_model.transform(test_data)

    # Evaluate using PySpark evaluator (accuracy)
    accuracy = evaluator.evaluate(predictions)
    print(f"{model_name} Accuracy: {accuracy:.4f}")

    # Extract predicted labels and actual labels from predictions DataFrame
    y_pred = np.array(predictions.select('prediction').rdd.flatMap(lambda x: x).collect())  # Extract predictions as numpy array
    y_true = np.array(test_data.select('label').rdd.flatMap(lambda x: x).collect())  # Convert true labels to numpy array

    # Calculate and print scikit-learn evaluation metrics
    print(f"F1 Score: {f1_score(y_true, y_pred, average='weighted'):.4f}")
    print(f"Precision: {precision_score(y_true, y_pred, average='weighted'):.4f}")
    print(f"Recall: {recall_score(y_true, y_pred, average='weighted'):.4f}")
    print(f"Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}")
    print("-" * 50)


Training Logistic Regression...
Logistic Regression Accuracy: 0.4211
F1 Score: 0.4253
Precision: 0.4342
Recall: 0.4211
Confusion Matrix:
[[7 0 2 4 0]
 [2 0 0 0 0]
 [1 0 1 0 0]
 [1 0 0 0 0]
 [1 0 0 0 0]]
--------------------------------------------------
Training Random Forest...


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Random Forest Accuracy: 0.8421
F1 Score: 0.8152
Precision: 0.8145
Recall: 0.8421
Confusion Matrix:
[[12  1  0  0  0]
 [ 0  2  0  0  0]
 [ 1  0  1  0  0]
 [ 0  0  0  1  0]
 [ 1  0  0  0  0]]
--------------------------------------------------
Training Decision Tree...


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Decision Tree Accuracy: 0.7895
F1 Score: 0.8035
Precision: 0.8676
Recall: 0.7895
Confusion Matrix:
[[10  1  0  2  0]
 [ 0  2  0  0  0]
 [ 1  0  1  0  0]
 [ 0  0  0  1  0]
 [ 0  0  0  0  1]]
--------------------------------------------------
